# Running Tune experiments with Nevergrad

In this tutorial we introduce Nevergrad, while running a simple Ray Tune experiment. Tune’s Search Algorithms integrate with Nevergrad and, as a result, allow you to seamlessly scale up a Nevergrad optimization process - without sacrificing performance.

Nevergrad provides gradient/derivative-free optimization able to handle noise over the objective landscape, including evolutionary, bandit, and Bayesian optimization algorithms. Nevergrad internally supports search spaces which are continuous, discrete or a mixture of thereof. It also provides a library of functions on which to test the optimization algorithms and compare with other benchmarks.

In this example we minimize a simple objective to briefly demonstrate the usage of Nevergrad with Ray Tune via `NevergradSearch`. It's useful to keep in mind that despite the emphasis on machine learning experiments, Ray Tune optimizes any implicit or explicit objective. Here we assume `nevergrad==0.4.3.post7` library is installed. To learn more, please refer to [Nevergrad website](https://github.com/facebookresearch/nevergrad).

In [1]:
# !pip install ray[tune]
!pip install nevergrad==0.4.3.post7 

  Using cached nevergrad-0.4.3.post7-py3-none-any.whl (400 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 3.2 MB/s eta 0:00:00a 0:00:01


Click below to see all the imports we need for this example.
You can also launch directly into a Binder instance to run this notebook yourself.
Just click on the rocket symbol at the top of the navigation.

In [2]:
import time

import ray
import nevergrad as ng
from ray import tune
from ray.air import session
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.nevergrad import NevergradSearch

/Users/kai/.pyenv/versions/3.7.7/lib/python3.7/site-packages/nevergrad/optimization/differentialevolution.py:107: InefficientSettingsWarning: DE algorithms are inefficient with budget < 60
  "DE algorithms are inefficient with budget < 60", base.errors.InefficientSettingsWarning


Let's start by defining a simple evaluation function.
We artificially sleep for a bit (`0.1` seconds) to simulate a long-running ML experiment.
This setup assumes that we're running multiple `step`s of an experiment and try to tune two hyperparameters,
namely `width` and `height`, and `activation`.

In [3]:
def evaluate(step, width, height, activation):
    time.sleep(0.1)
    activation_boost = 10 if activation=="relu" else 1
    return (0.1 + width * step / 100) ** (-1) + height * 0.1 + activation_boost

Next, our `objective` function takes a Tune `config`, evaluates the `score` of your experiment in a training loop,
and uses `session.report` to report the `score` back to Tune.

In [4]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"], config["activation"])
        session.report({"iterations": step, "mean_loss": score})

In [5]:
ray.init(configure_logging=False)

INFO:ray._private.services:View the Ray dashboard at http://127.0.0.1:8266


Python version:,3.7.7
Ray version:,3.0.0.dev0
Dashboard:,http://127.0.0.1:8266


Now we construct the hyperparameter search space using `ConfigSpace`

Next we define the search algorithm built from `NevergradSearch`, constrained  to a maximum of `4` concurrent trials with a `ConcurrencyLimiter`. Here we use `ng.optimizers.OnePlusOne`, a simple evolutionary algorithm.

In [6]:
algo = NevergradSearch(
    optimizer=ng.optimizers.OnePlusOne,
)
algo = tune.search.ConcurrencyLimiter(algo, max_concurrent=4)

The number of samples is the number of hyperparameter combinations that will be tried out. This Tune run is set to `1000` samples.
(you can decrease this if it takes too long on your machine).

In [7]:
num_samples = 1000

In [8]:
# If 1000 samples take too long, you can reduce this number.
# We override this number here for our smoke tests.
num_samples = 10

Finally, all that's left is to define a search space.

In [9]:
search_config = {
    "steps": 100,
    "width": tune.uniform(0, 20),
    "height": tune.uniform(-100, 100),
    "activation": tune.choice(["relu, tanh"])
}

Finally, we run the experiment to `"min"`imize the "mean_loss" of the `objective` by searching `search_space` via `algo`, `num_samples` times. This previous sentence is fully characterizes the search problem we aim to solve. With this in mind, observe how efficient it is to execute `tuner.fit()`.

In [10]:
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        num_samples=num_samples,
    ),
    param_space=search_config,
)
results = tuner.fit()

Trial name,status,loc,activation,height,width,loss,iter,total time (s),iterations,neg_mean_loss
objective_ee2ca136,TERMINATED,127.0.0.1:46434,"relu, tanh",0,10,1.1,100,10.942,99,-1.1
objective_efe1626e,TERMINATED,127.0.0.1:46441,"relu, tanh",-31.0013,9.28761,-1.99254,100,11.5354,99,1.99254
objective_efe34e4e,TERMINATED,127.0.0.1:46442,"relu, tanh",5.21403,9.48974,1.62672,100,11.6606,99,-1.62672
objective_efe55c2a,TERMINATED,127.0.0.1:46443,"relu, tanh",-20.8721,11.3958,-0.99935,100,11.6083,99,0.99935
objective_f6688086,TERMINATED,127.0.0.1:46467,"relu, tanh",57.2829,17.7296,6.78493,100,10.716,99,-6.78493
objective_f85ed926,TERMINATED,127.0.0.1:46478,"relu, tanh",40.5543,19.0813,5.10809,100,10.7158,99,-5.10809
objective_f86ee276,TERMINATED,127.0.0.1:46481,"relu, tanh",93.8686,1.60757,10.9781,100,10.7415,99,-10.9781
objective_f880a02e,TERMINATED,127.0.0.1:46484,"relu, tanh",-80.5769,5.84852,-6.88791,100,10.7335,99,6.88791
objective_fe4e7a44,TERMINATED,127.0.0.1:46499,"relu, tanh",9.62911,0.622909,3.35823,100,13.1428,99,-3.35823
objective_004f499a,TERMINATED,127.0.0.1:46504,"relu, tanh",-90.5065,2.11741,-7.59533,100,10.7688,99,7.59533


INFO:ray._private.runtime_env.plugin_schema_manager:Loading the default runtime env schemas: ['/Users/kai/coding/ray/python/ray/_private/runtime_env/../../runtime_env/schemas/working_dir_schema.json', '/Users/kai/coding/ray/python/ray/_private/runtime_env/../../runtime_env/schemas/pip_schema.json'].


Result for objective_ee2ca136:
  date: 2022-07-22_15-24-03
  done: false
  experiment_id: c0ad5ddb78cc4cc88e8195f5bde34e20
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 11.0
  neg_mean_loss: -11.0
  node_ip: 127.0.0.1
  pid: 46434
  time_since_restore: 0.10390329360961914
  time_this_iter_s: 0.10390329360961914
  time_total_s: 0.10390329360961914
  timestamp: 1658499843
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: ee2ca136
  warmup_time: 0.003058910369873047
  
Result for objective_efe1626e:
  date: 2022-07-22_15-24-06
  done: false
  experiment_id: c6d33a9a30c040c0929b657f1d3e1557
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 7.899868000941147
  neg_mean_loss: -7.899868000941147
  node_ip: 127.0.0.1
  pid: 46441
  time_since_restore: 0.10202908515930176
  time_this_iter_s: 0.10202908515930176
  time_total_s: 0.10202908515930176
  timestamp: 1658499846
  timesteps_since_r

INFO:ray.tune.tune:Total run time: 44.70 seconds (43.68 seconds for the tuning loop).


Here are the hyperparamters found to minimize the mean loss of the defined objective.

In [11]:
print("Best hyperparameters found were: ", results.get_best_result().config)

Best hyperparameters found were:  {'steps': 100, 'width': 2.1174116156230918, 'height': -90.50653873694615, 'activation': 'relu, tanh'}


## Optional: passing the (hyper)parameter space into the search algorithm

We can also pass the search space into `NevergradSearch` using their designed format.

In [12]:
space = ng.p.Dict(
    width=ng.p.Scalar(lower=0, upper=20),
    height=ng.p.Scalar(lower=-100, upper=100),
    activation=ng.p.Choice(choices=["relu", "tanh"])
)

In [13]:
algo = NevergradSearch(
    optimizer=ng.optimizers.OnePlusOne,
    space=space,
    metric="mean_loss",
    mode="min"
)
algo = tune.search.ConcurrencyLimiter(algo, max_concurrent=4)

Again we run the experiment, this time with a less passed via the `config` and instead passed through `search_alg`.

In [14]:
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
#         metric="mean_loss",
#         mode="min",
        search_alg=algo,
        num_samples=num_samples,
    ),
    param_space={"steps": 100},
)
results = tuner.fit()

Trial name,status,loc,activation,height,width,loss,iter,total time (s),iterations,neg_mean_loss
objective_085274be,TERMINATED,127.0.0.1:46516,tanh,0,10,1.1,100,10.7324,99,-1.1
objective_09dee4f2,TERMINATED,127.0.0.1:46524,tanh,-44.4216,12.9653,-3.36485,100,11.3476,99,3.36485
objective_09e0846a,TERMINATED,127.0.0.1:46525,relu,-38.0638,11.1574,6.28334,100,11.3103,99,-6.28334
objective_09e21122,TERMINATED,127.0.0.1:46526,relu,41.2509,9.75585,14.2276,100,11.3512,99,-14.2276
objective_1045958e,TERMINATED,127.0.0.1:46544,relu,-73.2818,5.78832,2.84334,100,10.7372,99,-2.84334
objective_12309db2,TERMINATED,127.0.0.1:46549,relu,-94.9666,16.9764,0.562486,100,10.7329,99,-0.562486
objective_12342770,TERMINATED,127.0.0.1:46550,tanh,-98.0775,17.2252,-8.74945,100,10.7455,99,8.74945
objective_12374d7e,TERMINATED,127.0.0.1:46551,relu,-1.60759,18.0841,9.89479,100,10.7348,99,-9.89479
objective_18344524,TERMINATED,127.0.0.1:46569,tanh,-41.1284,12.2952,-3.03135,100,12.622,99,3.03135
objective_1a1e29b8,TERMINATED,127.0.0.1:46576,tanh,64.0289,10.0482,7.50242,100,10.8237,99,-7.50242


Result for objective_085274be:
  date: 2022-07-22_15-24-47
  done: false
  experiment_id: be25590fb790400ca28b548b8359a120
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 11.0
  neg_mean_loss: -11.0
  node_ip: 127.0.0.1
  pid: 46516
  time_since_restore: 0.10224103927612305
  time_this_iter_s: 0.10224103927612305
  time_total_s: 0.10224103927612305
  timestamp: 1658499887
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 085274be
  warmup_time: 0.0029327869415283203
  
Result for objective_09dee4f2:
  date: 2022-07-22_15-24-49
  done: false
  experiment_id: 9469ffc8df11476db00e329acd2ae8b6
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 6.557843114006006
  neg_mean_loss: -6.557843114006006
  node_ip: 127.0.0.1
  pid: 46524
  time_since_restore: 0.10509586334228516
  time_this_iter_s: 0.10509586334228516
  time_total_s: 0.10509586334228516
  timestamp: 1658499889
  timesteps_since_

INFO:ray.tune.tune:Total run time: 43.33 seconds (43.21 seconds for the tuning loop).


Here are the hyperparamters found to minimize the mean loss of the defined objective. Note that we have to pass the metric and mode here because we don't set it in the TuneConfig.

In [17]:
print("Best hyperparameters found were: ", results.get_best_result("mean_loss", "min").config)

Best hyperparameters found were:  {'steps': 100, 'width': 17.225166732233465, 'height': -98.07750812064515, 'activation': 'tanh'}


In [15]:
ray.shutdown()